In [1]:
from RootInteractive.Tools.histoNDTools import *

Welcome to JupyROOT 6.15/02


# Random Data Frame generator

In [2]:
npoints=10000
df = pd.DataFrame(np.random.random_sample(size=(npoints, 4)), columns=list('ABCD'))
df["noise"]=np.random.normal(0, 0.1, npoints)
df["csin"]=np.sin(6.28*df["C"])
df["valueOrig"]=df["A"]+np.exp(3*df["B"])*df["csin"]
df["value"]=df["valueOrig"]+df["noise"]
df.tail()


,A,B,C,D,noise,csin,valueOrig,value
9995,0.675872,0.898350,0.672147,0.396875,-0.097730,-0.881716,-12.379060,-12.476790
9996,0.656222,0.559829,0.164708,0.491056,-0.059501,0.859538,5.265761,5.206260
9997,0.530604,0.919275,0.149171,0.721606,-0.007917,0.805664,13.232302,13.224385
9998,0.447976,0.679730,0.753563,0.550502,-0.011271,-0.999800,-7.234872,-7.246144
9999,0.168503,0.825680,0.839731,0.118031,0.125683,-0.846658,-9.911788,-9.786105


# Make Histogram

In [3]:
histo=makeHistogram(df,"value:A:B:C:D:#A>0>>myhisto(50,-25,25,40,0,1,30,0,1,20,0,1,10,0,1)")


# Inputs

In [4]:
slices=((0,50,1,0),(10,30,1,4),(10,24,5,3),(10,16,2,2),(0,10,3,0))   # start:stop:step:width
dimI=0                                                               # dimension of interest  

# The Code of the function

### Width modification

In [5]:
binList = []                                # Prepare list of bin numbers which has same dimensions with original histogram
for el in histo["axes"]:                    #     so same slicing can be applied on the bin list
    binList.append(np.arange(len(el)-1))

    
widthList=[item[3] for item in slices]      # extract the list of widths from slicing tuple
localHist = histo["H"].copy()
localAxes = histo["axes"].copy()


for iDim, w in enumerate(widthList):          # get the original histogram and shifted histograms by +/- i,
    widthSliceList=[]                         #     where i is all integers upto width, sum all histograms 
    L=localHist.shape[iDim]
    
    iSlice=[slice(None)]*len(localHist.shape)
    iSlice[iDim]=slice(w,L-w)
    dummy=localHist[tuple(iSlice)] 
    
    for i in range(1,w+1):
        iSlice=[slice(None)]*len(localHist.shape)
        iSlice[iDim]=slice(w+i,L-w+i)
        dummy+=localHist[tuple(iSlice)]
        
        iSlice=[slice(None)]*len(localHist.shape)
        iSlice[iDim]=slice(w-i,L-w-i)
        dummy+=localHist[tuple(iSlice)]
        
    localHist = dummy
    binList[iDim] = binList[iDim][slice(w,L-w)]    # trim the edges
    localAxes[iDim] = localAxes[iDim][slice(w,L-w+1)]

print(widthList)
print(localHist.shape)
for a, b in zip(localAxes,binList):
    print(a.shape, b.shape)
print(binList)


[0, 4, 3, 2, 0]
(50, 32, 24, 16, 10)
(51,) (50,)
(33,) (32,)
(25,) (24,)
(17,) (16,)
(11,) (10,)
[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]), array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]), array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26]), array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]), array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])]


### Slicing tuple

In [6]:
newSliceList=[]
for iDim, iSlice in enumerate(slices):    # rearranging slice accourding to width
    jSlice=slice(iSlice[0] - iSlice[3], iSlice[1] - iSlice[3], iSlice[2])
        
    if jSlice.start < 0 or jSlice.stop >  localHist.shape[iDim]:
        raise ValueError("Range and width of {}th dimension is not compatible: {}".format(iDim,iSlice))
    
    newSliceList.append(jSlice)  
    
    
npSlice=tuple(newSliceList)


print(histo["H"].shape)
print(localHist.shape)
print(slices)
print(newSliceList)
print(npSlice)
print(npAxesSlice)
for a,b in zip(histo["axes"],localAxes):
    print(a.shape, b.shape)

(50, 40, 30, 20, 10)
(50, 32, 24, 16, 10)
((0, 50, 1, 0), (10, 30, 1, 4), (10, 24, 5, 3), (10, 16, 2, 2), (0, 10, 3, 0))
[slice(0, 50, 1), slice(6, 26, 1), slice(7, 21, 5), slice(8, 14, 2), slice(0, 10, 3)]
(slice(0, 50, 1), slice(6, 26, 1), slice(7, 21, 5), slice(8, 14, 2), slice(0, 10, 3))
(slice(0, 51, 1), slice(6, 27, 1), slice(7, 26, 5), slice(8, 16, 2), slice(0, 13, 3))
(51,) (51,)
(41,) (33,)
(31,) (25,)
(21,) (17,)
(11,) (11,)


### Determine the bin centers

In [7]:
CenterList = []
for arr in localAxes:
    arri = arr[:-1] + (arr[1]-arr[0])/2
    CenterList.append(arri)
    
    
print(CenterList)

[array([-24.5, -23.5, -22.5, -21.5, -20.5, -19.5, -18.5, -17.5, -16.5,
       -15.5, -14.5, -13.5, -12.5, -11.5, -10.5,  -9.5,  -8.5,  -7.5,
        -6.5,  -5.5,  -4.5,  -3.5,  -2.5,  -1.5,  -0.5,   0.5,   1.5,
         2.5,   3.5,   4.5,   5.5,   6.5,   7.5,   8.5,   9.5,  10.5,
        11.5,  12.5,  13.5,  14.5,  15.5,  16.5,  17.5,  18.5,  19.5,
        20.5,  21.5,  22.5,  23.5,  24.5]), array([0.1125, 0.1375, 0.1625, 0.1875, 0.2125, 0.2375, 0.2625, 0.2875,
       0.3125, 0.3375, 0.3625, 0.3875, 0.4125, 0.4375, 0.4625, 0.4875,
       0.5125, 0.5375, 0.5625, 0.5875, 0.6125, 0.6375, 0.6625, 0.6875,
       0.7125, 0.7375, 0.7625, 0.7875, 0.8125, 0.8375, 0.8625, 0.8875]), array([0.11666667, 0.15      , 0.18333333, 0.21666667, 0.25      ,
       0.28333333, 0.31666667, 0.35      , 0.38333333, 0.41666667,
       0.45      , 0.48333333, 0.51666667, 0.55      , 0.58333333,
       0.61666667, 0.65      , 0.68333333, 0.71666667, 0.75      ,
       0.78333333, 0.81666667, 0.85      , 0.883333

### aply the npslice

In [8]:
histogram = localHist[npSlice]
localCenter = []
localNumbers = []

for i,(b,c) in enumerate(zip(binList,CenterList)):
    localNumbers.append(b[npSlice[i]])
    localCenter.append(c[npSlice[i]])
    
print(localNumbers)
print(localCenter)

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]), array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29]), array([10, 15, 20]), array([10, 12, 14]), array([0, 3, 6, 9])]
[array([-24.5, -23.5, -22.5, -21.5, -20.5, -19.5, -18.5, -17.5, -16.5,
       -15.5, -14.5, -13.5, -12.5, -11.5, -10.5,  -9.5,  -8.5,  -7.5,
        -6.5,  -5.5,  -4.5,  -3.5,  -2.5,  -1.5,  -0.5,   0.5,   1.5,
         2.5,   3.5,   4.5,   5.5,   6.5,   7.5,   8.5,   9.5,  10.5,
        11.5,  12.5,  13.5,  14.5,  15.5,  16.5,  17.5,  18.5,  19.5,
        20.5,  21.5,  22.5,  23.5,  24.5]), array([0.2625, 0.2875, 0.3125, 0.3375, 0.3625, 0.3875, 0.4125, 0.4375,
       0.4625, 0.4875, 0.5125, 0.5375, 0.5625, 0.5875, 0.6125, 0.6375,
       0.6625, 0.6875, 0.7125, 0.7375]), array([0.35      , 0.51666667, 0.683333

# Bin Number

In [9]:
mesharrayNumbers = np.array(np.meshgrid(*localNumbers, indexing='ij'))

In [10]:
binNumbers = []
for el in mesharrayNumbers:
    binNumbers.append(el.flatten())

# Bin Center

In [11]:
mesharrayCenter = np.array(np.meshgrid(*localCenter, indexing='ij'))

In [12]:
binCenters = []
for el in mesharrayCenter:
    binCenters.append(el.flatten())

In [17]:
histogramMap={}
varNames=histo['varNames']

for iDim in range(len(varNames)):
    if iDim is dimI:
        continue
    histogramMap[varNames[iDim] +"BinNumber"] = binNumbers[iDim]
    histogramMap[varNames[iDim] +"BinCenter"] = binCenters[iDim]
    print(len(binNumbers[iDim]))
    print(len(binCenters[iDim]))

means = np.ndarray.flatten(np.mean(histogram,axis=dimI))
rms = np.ndarray.flatten(np.sqrt(np.mean(histogram**2,axis=dimI)))
medians = np.ndarray.flatten(np.median(histogram,axis=dimI))
entries = np.ndarray.flatten(np.sum(histogram,axis=dimI))
print(means.shape)

histogramMap["means"] = means
histogramMap["rms"] = rms
histogramMap["medians"] = medians
histogramMap["entries"] = entries
out=pd.DataFrame(histogramMap)


36000
36000
36000
36000
36000
36000
36000
36000
(720,)


ValueError: arrays must all be same length

In [ ]:
out.head